<a href="https://colab.research.google.com/github/ChangeIsKey/NovelSenseDiscovery/blob/main/load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ares
import requests
url ="http://sensembert.org/resources/ares_embedding.tar.gz" 
r = requests.get(url)

In [ ]:
# sensebert sample
!rm -r NovelSenseDiscovery
!git clone https://github.com/ChangeIsKey/NovelSenseDiscovery

#!cd NovelSenseDiscovery/data/ && tar -xf sensembert_data.tar.gz
#!ls NovelSenseDiscovery/data/sensembert_data 
#sense_vectors = 'NovelSenseDiscovery/data/sensembert_data/sensembert_EN_supervised.txt'

!cd NovelSenseDiscovery/data/ && tar -xf ares_embedding.tar.gz
!ls NovelSenseDiscovery/data/ares_embedding
sense_vectors = 'NovelSenseDiscovery/data/ares_embedding/ares_bert_base_multilingual.txt'

In [ ]:
# get discovery repo
!rm -r LSCDiscovery
!git clone https://github.com/seinan9/LSCDiscovery
!pip install -r LSCDiscovery/requirements.txt
!python -m spacy download en_core_web_sm
#!pip install --force-reinstall -v "spacy==3.1.1"

In [ ]:
#load vectors
#!git clone https://github.com/Garrafao/LSCDetection
#from gensim.models import KeyedVectors
#sense_vecs = KeyedVectors.load_word2vec_format(sense_vectors, binary=False)
from LSCDiscovery.modules.utils_ import Space

# Load matrices
try: 
    space = Space(sense_vectors, format='npz') 
except ValueError: 
    space = Space(sense_vectors, format='w2v')

sense_vecs = space.matrix.toarray()
#split vectors
sense_vecs = sense_vecs[:,:1024] #use synset embedding
#sense_vecs = sense_vecs[1024:] #use sense embedding
row2id_senses = space.row2id
id2row_senses = space.id2row

#display(sense_vecs)
#display(row2id_senses)
#display(sense_vecs.shape)

In [ ]:
# get benchmark
#!git clone https://github.com/ChangeIsKey/LSCDBenchmark
#!pip install -r LSCDBenchmark/requirements.txt
#!pip install torch

In [ ]:
#import sys 
#sys.path.append('./LSCDBenchmark/') 
#import main

In [ ]:
# download semeval corpora
!cd LSCDiscovery && bash scripts/import_semeval_en.sh
#!ls data

In [ ]:
# sample uses
!cd LSCDiscovery && bash -e scripts/prepare_sample.sh en_semeval sample_1 10 25 en

In [ ]:
#intersect sense and corpus vocab

In [ ]:
# extract usage vectors 
!cd LSCDiscovery && bash scripts/discover_bert.sh en_semeval sample_1 en-large toklem 1+12 0.1

In [ ]:
#extract wsd data
!cd NovelSenseDiscovery/data/ && tar -xf data_files.tar.gz
#!ls NovelSenseDiscovery/data/data_files

In [ ]:
# visualize targets
#!ls LSCDiscovery/data/en_semeval/samples/sample_1
#targets_path = 'LSCDiscovery/data/en_semeval/samples/sample_1/sample.tsv'
targets_path = 'NovelSenseDiscovery/data/all.tsv'

import csv

with open(targets_path, encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile,fieldnames=['target'], delimiter='\t',quoting=csv.QUOTE_NONE,strict=True)
    targets = [row['target'] for row in reader]

#display(targets)

#uses_path = 'LSCDiscovery/data/en_semeval/samples/sample_1/usages_corpus1/casket.tsv'
#uses_path = 'NovelSenseDiscovery/data/data_files/1970s.tsv'
#!ls LSCDiscovery/data/en_semeval/samples/sample_1/usages_corpus1
# Uses
#with open(uses_path, encoding='utf-8') as csvfile:
#    reader = csv.DictReader(csvfile, delimiter='\t',quoting=csv.QUOTE_NONE,strict=True)
#    rows = [row for row in reader]
#display(rows)

In [ ]:
#vectorize wsd test usages
%%bash
files=NovelSenseDiscovery/data/data_files
outputdir=NovelSenseDiscovery/data/data_files_vectors
rm -r $outputdir
mkdir $outputdir
cd LSCDiscovery
for file in ../$files/*.tsv
do
    echo $file
    python contextualized/bert.py -l $file ../$outputdir/$(basename "${file%.*}") en-large toklem 1+12
    #break #for test
done

In [ ]:
# merge

In [ ]:
#load vectors for each target and assign nearest neighbor sense
#!ls LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors_corpus1

import numpy
import csv
from sklearn.neighbors import NearestNeighbors 
#vecs_path = 'LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors1'
#vecs_path = 'LSCDiscovery/output/en_semeval/BERT_layers1+12_typetoklem/discovery/sample_1/t0.1/vectors2'
vecs_path = 'NovelSenseDiscovery/data/data_files_vectors'
#output_file = 'predictions_coha.csv'
output_file = 'predictions_data_files.csv'
usage2sensepred = {}
for target in targets:
    display(target)
        
    path_matrix1 = vecs_path+'/'+target
    
    # Load matrices
    try:          
        try: 
            space1 = Space(path_matrix1, format='npz') 
        except ValueError:
            space1 = Space(path_matrix1, format='w2v') 
    except FileNotFoundError:        
         continue

    row2id1 = space1.row2id

    #display(row2id1)

    vectors1 = space1.matrix.toarray() 
    usage_vecs_no = vectors1.shape[0]
    vecs = numpy.vstack((vectors1,sense_vecs))
    vecs_no = vecs.shape[0]
    #row2id = {}
    #for (row,id) in row2id1 | row2id2 | row2id_sense:
    #    row2id[row] = row

    #Get nearest neighbor sense
    nbrs = NearestNeighbors(n_neighbors=vecs_no, algorithm='auto', metric='cosine').fit(vecs) 
    distances, indices = nbrs.kneighbors(vecs) 
    #display(indices)
    for i in range(indices.shape[0]):
        inds = indices[i]
        for nbr in inds:
            if nbr > usage_vecs_no: #todo validate
                usage2sensepred[target+'_'+str(i-usage_vecs_no)] = id2row_senses[nbr-usage_vecs_no] #todo add usage id
                break
        #break #shorten list for test 

display(usage2sensepred)

predictions = [{'usage_id': u, 'sense_id':s} for (u,s) in usage2sensepred.items()]
# Export predictions
with open(output_file, 'w') as f:  
    w = csv.DictWriter(f, predictions[0].keys(), delimiter='\t', quoting = csv.QUOTE_NONE, quotechar='')
    w.writeheader()
    w.writerows(predictions)